In [ ]:
import pandas as pd
import numpy as np
import sys,os
from collections import Counter

In [ ]:
!unzip /content/drive/MyDrive/initial_data.zip

Archive:  /content/drive/MyDrive/initial_data.zip
   creating: initial_data/NORMAL_PERMITS/
  inflating: initial_data/NORMAL_PERMITS/Vevaioseis-APE.xlsx  
  inflating: initial_data/NORMAL_PERMITS/ЛЖТРЧО-АГДИЧМ-ДРВА-АПД-OKTЧБРИОС-2023.xlsx  
  inflating: initial_data/NORMAL_PERMITS/ЛЖТРЧО-АГДИЧМ-ДРВА-АПД-ИАМОУАРИОС-2023.xlsx  
  inflating: initial_data/NORMAL_PERMITS/ЛЖТРЧО-АГДИЧМ-ДРВА-АПД-ИОУКИОС-2023.xlsx  
  inflating: initial_data/NORMAL_PERMITS/ЛЖТРЧО-АГДИЧМ-ДРВА-АПД-ИОУМИОС-2023.xlsx  
  inflating: initial_data/NORMAL_PERMITS/ЛЖТРЧО-АГДИЧМ-ДРВА-АПД-ЛАIOС-2023.xlsx  
  inflating: initial_data/NORMAL_PERMITS/ЛЖТРЧО-АГДИЧМ-ДРВА-АПД-ЛАРТИОС-2023.xlsx  
  inflating: initial_data/NORMAL_PERMITS/ЛЖТРЧО-АГДИЧМ-ДРВА-АПД-ЛАРТИОС-2024.xlsx  
  inflating: initial_data/NORMAL_PERMITS/ЛЖТРЧО-АГДИЧМ-ДРВА-АПД-МОДЛБРИОС-2023.xlsx  
  inflating: initial_data/NORMAL_PERMITS/ЛЖТРЧО-АГДИЧМ-ДРВА-АПД-СДПТДЛБРИОС-2023-1.xlsx  
  inflating: initial_data/NORMAL_PERMITS/ЛЖТРЧО-АГДИЧМ-ДРВА-АПД-ФДБРОУАРИОС-20

In [ ]:
# !unzip /content/drive/MyDrive/MARIOS_data.zip

In [ ]:
def rename_cols(df):
    df_copy = df.copy()
    for col in  df_copy.columns:
        if col == 'ΙΣΧΥΣ (MW)' :
            df_copy.rename(columns={col:'ΜΕΓΙΣΤΗ ΙΣΧΥΣ (MW)'},inplace= True)
        if col ==  'ΑΡ. ΒΕΒΑΙΩΣΗΣ ΡΑΕ':
            df_copy.drop(columns=['ΑΡ. ΒΕΒΑΙΩΣΗΣ ΡΑΕ'],inplace=True)
        if col ==  'ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ':
            df_copy.rename(columns={col:'AΡΙΘΜΟΣ ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ'},inplace= True)

    return df_copy

In [ ]:
NORMAL_PERMITS_path = './initial_data/NORMAL_PERMITS'
SPECIAL_PERMIT_path = './initial_data/SPECIAL_PERMITS'

path_list=[SPECIAL_PERMIT_path,NORMAL_PERMITS_path]
# Για εργασία page exploratory
columns_list_unique,tail_permits,count_nan_columns_list = [],[],[]
length_per_file_list,name_file_list,drop_bebaiosi_rae =[],[],[]
# End of page exploratory
c=0
# collect_column_names =[]
# cols_number_list =[]
df_list=[]
summa=0

for p in path_list:
    for files in os.listdir(p):
        print(files)
        flag_self_productor = False
        flag_contains = False
        #
        name_file_list.append(files)

        temp_df = pd.read_excel(f'{p}/{files}',sheet_name='ΑΔΕΙΕΣ ΑΠΕ')
        temp_df.columns = temp_df.iloc[0] # take the first row
        columns_with_nan_names = [col for col in temp_df.columns if col!=col]
        count_nan_columns_list.append(len(columns_with_nan_names))

    #     print(len(columns_with_nan_names))
        if len(columns_with_nan_names)!=0:
            temp_df.drop(columns_with_nan_names,axis=1,inplace=True)

        temp_df = temp_df.drop(0) # Drop the big starting title
        temp_df.columns.name = None #Erase the index for columns beecause previously they were rows
        temp_df.reset_index(inplace=True,drop=True) #Reset the index

        if temp_df.tail(1).iloc[:,0].to_string(index=False)=='** Αυτοπαραγωγός': # Tελευταίο στοιχείο έχει στοιχείο αυτοπαραγωγός
#             print('check this file')
            # tail_permits.append(temp_df.tail(1).iloc[:,0].to_string(index=False))
            flag_self_productor = True
            temp_df.drop(temp_df.tail(1).index,inplace=True)

        if 'Δεν συμπεριλαμβάνονται' in temp_df.tail(1).iloc[:,0].to_string(index=False):
            #print('check double this file')
            flag_contains = True
            # tail_permits.append(temp_df.tail(1).iloc[:,0].to_string(index=False))
            temp_df.drop(temp_df.tail(1).index,inplace=True)
        temp_df.reset_index(inplace=True,drop=True) #Reset the index
        #1 Δεν συμπεριλαμβάνονται υβριδικοί σταθμοί #2 ** Αυτοπαραγωγός
        if flag_self_productor and flag_contains:
          tail_permits.append('** Αυτοπαραγωγός , Δεν συμπεριλαμβάνονται υβριδικοί σταθμοί')
        elif flag_self_productor:
          tail_permits.append('** Αυτοπαραγωγός')
        elif flag_contains:
          tail_permits.append('Δεν συμπεριλαμβάνονται υβριδικοί σταθμοί')
        else:
          tail_permits.append(' ')


        columns_list_unique.append(','.join(temp_df.columns))
        drop_bebaiosi_rae.append(f"Total:{len(temp_df)} Nan:{temp_df['ΑΡ. ΒΕΒΑΙΩΣΗΣ ΡΑΕ'].isna().sum()}" if col=='ΑΡ. ΒΕΒΑΙΩΣΗΣ ΡΑΕ' else 'NOT EXIST' for col in temp_df.columns if col=='ΑΡ. ΒΕΒΑΙΩΣΗΣ ΡΑΕ')
        f_temp_df = rename_cols(temp_df.copy())
        print('Number of rows',len(f_temp_df),'Number of columns',len(f_temp_df.columns))
        summa += len(f_temp_df)
#         print(len(f_temp_df.columns))
        if len(f_temp_df.columns)<15:
            df_list.append(f_temp_df)
            length_per_file_list.append(len(f_temp_df))

М.4685-ЛЖТРЧО-АГДИЧМ-ДИГИЙА-ДРВА-АПД-ГДЙДЛБРИОС-2022.xlsx
Number of rows 198 Number of columns 13
М.4685-ЛЖТРЧО-АГДИЧМ-ДИГИЙА-ДРВА-АПД-ЛАЇОС-2021.xlsx
Number of rows 182 Number of columns 13
ЛЖТРЧО-АГДИЧМ-ДИГИЙА-ДРВА-АПД-СДПТДЛБРИОС-2023.xlsx
Number of rows 206 Number of columns 13
ЛЖТРЧО-АГДИЧМ-ДИГИЙА-ДРВА-АПД-ОЙТЧБРИОС-2023.xlsx
Number of rows 206 Number of columns 13
1220s.xlsx
Number of rows 184 Number of columns 13
ЛЖТРЧО-АГДИЧМ-ДИГИЙА-ДРВА-АПД-ИАМОУАРИОС-2023.xlsx
Number of rows 206 Number of columns 13
М.4685-ЛЖТРЧО-АГДИЧМ-ДИГИЙА-ДРВА-АПД-СДПТДЛБРИОС-2022.xlsx
Number of rows 198 Number of columns 13
ЛЖТРЧО-АГДИЧМ-ДИГИЙА-ДРВА-АПД-ФДБРОУАРИОС-2024.xlsx
Number of rows 200 Number of columns 13
ЛЖТРЧО-АГДИЧМ-ДИГИЙА-ДРВА-АПД-ФДБРОУАРИОС-2023.xlsx
Number of rows 205 Number of columns 13
М.4685-ЛЖТРЧО-АГДИЧМ-ДИГИЙА-ДРВА-АПД-АУВОУСТОС-2021.xlsx
Number of rows 182 Number of columns 13
ЛЖТРЧО-АГДИЧМ-ДИГИЙА-ДРВА-АПД-ЛАРТИОС-2024.xlsx
Number of rows 198 Number of columns 13
ЛЖТРЧО-АГДИЧМ-ДИГ

In [ ]:
# from pprint import pprint
# pprint(drop_bebaiosi_rae)

In [ ]:
# stringa_1 = ','.join(['ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ',' ΑΙΤΗΣΗ', 'ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ', 'ΕΤΑΙΡΕΙΑ', 'ΔΗΜΟΣ' , 'ΤΕΧΝΟΛΟΓΙΑ',' ΠΕΡΙΦΕΡΕΙΑ', 'ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ', 'ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ', 'ΙΣΧΥΣ (MW)', 'ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ', 'ΘΕΣΗ', 'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'])
# print(stringa_1)


In [ ]:
print(len(columns_list_unique),len(tail_permits),len(count_nan_columns_list),len(length_per_file_list),len(name_file_list),len(drop_bebaiosi_rae))

38 38 38 38 38 38


In [ ]:
# Datasets data frame
#Only for the second project

step_1_exploratory = {
    'file name':name_file_list,
    'total rows':length_per_file_list,
    'tail values':tail_permits,
    'number nan cols':count_nan_columns_list ,
    'drop bebaiosi RAE':drop_bebaiosi_rae,

}
columns_list = [list(set(list(c))) for c in columns_list_unique]

step_2_exploratory = {
    'file name':name_file_list,
    'file_columns':columns_list_unique
}

step_1_df = pd.DataFrame(data=step_1_exploratory)
step_2_df = pd.DataFrame(data=step_2_exploratory)

In [ ]:
print(step_2_df.to_string())

                                                     file name                                                                                                                                                                                                                                file_columns
0    М.4685-ЛЖТРЧО-АГДИЧМ-ДИГИЙА-ДРВА-АПД-ГДЙДЛБРИОС-2022.xlsx                            ΑΙΤΗΣΗ,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΙΣΧΥΣ (MW),ΤΕΧΝΟΛΟΓΙΑ
1         М.4685-ЛЖТРЧО-АГДИЧМ-ДИГИЙА-ДРВА-АПД-ЛАЇОС-2021.xlsx                            ΑΙΤΗΣΗ,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,ΑΡ. ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΡΑΕ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΙΣΧΥΣ (MW),ΤΕΧΝΟΛΟΓΙΑ
2          ЛЖТРЧО-АГДИЧМ-ДИГИЙА-ДРВА-АПД-СДПТДЛБРИОС-2023.xlsx                            ΑΙΤΗΣΗ,ΗΜΕΡΟΜ

In [ ]:
step_1_df.to_csv('./exploratory_step_1.csv')
step_2_df.to_csv('./exploratory_step_2.csv')

In [ ]:
print('TOTAL DATASET LINES',summa)

TOTAL DATASET LINES 98095


In [ ]:
cols_name_list =[]
for el in df_list:
    cols_name_list.extend(el.columns.tolist())

# Dataframe το ένα κάτω απο το άλλο

In [ ]:
pre_final_df = pd.concat(df_list,axis=0)
pre_final_df.reset_index(inplace=True,drop=True)
pre_final_df.to_csv('./exploratory_step_3.csv')

In [ ]:
pre_final_df

,ΑΙΤΗΣΗ,ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ,ΕΤΑΙΡΕΙΑ,AΡΙΘΜΟΣ ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ,ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ,ΠΕΡΙΦΕΡΕΙΑ,ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ,ΔΗΜΟΣ,ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ,ΘΕΣΗ,ΜΕΓΙΣΤΗ ΙΣΧΥΣ (MW),ΤΕΧΝΟΛΟΓΙΑ
0,Β-00279,2001-02-19 00:00:00,ΓΕΩΘΕΡΜΙΚΟΣ ΣΤΟΧΟΣ ΔΥΟ ΜΟΝΟΠΡΟΣΩΠΗ Α.Ε.,ΑΔ-00488,2003-03-04 00:00:00,2028-03-04 00:00:00,Β ΑΙΓΑΙΟΥ,ΛΕΣΒΟΥ,ΛΕΣΒΟΥ,ΠΕΤΡΑΣ,ΣΤΥΨΗ/ΠΕΡΙΟΧΗ ΑΡΓΕΝΟΥ,8,ΓΕΩΘΕΡΜΙΑ
1,Γ-00798,2004-11-18 00:00:00,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,ΑΔ-00911,2006-03-23 00:00:00,2031-03-23 00:00:00,ΑΤΤΙΚΗΣ,ΑΤΤΙΚΗΣ,ΛΑΥΡΕΩΤΙΚΗΣ,ΛΑΥΡΕΩΤΙΚΗΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ Ι,45,ΑΙΟΛΙΚΑ
2,Γ-01329,2006-01-24 00:00:00,ENEL GREEN POWER HELLAS A.E.,ΑΔ-00981,2006-10-02 00:00:00,2031-10-02 00:00:00,ΣΤ ΕΛΛΑΔΑΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΚΑΡΥΣΤΟΥ ΚΑΙ ΚΟΙΝΟΤΗΤΑ ΚΑΦΗΡΕΩΣ,ΠΑΛΙΟΠΥΡΓΟΣ,13.8,ΑΙΟΛΙΚΑ
3,Γ-01089,2005-07-22 00:00:00,ΤΕΡΝΑ ΕΝΕΡΓΕΙΑΚΗ ΑΙ ΓΙΩΡΓΗΣ ΑΕ,ΑΔ-00989,2006-10-16 00:00:00,2031-10-16 00:00:00,ΑΤΤΙΚΗΣ,ΑΤΤΙΚΗΣ,ΥΔΡΑΣ,ΥΔΡΑΣ,ΝΗΣΙΔΑ ΑΓΙΟΣ ΓΕΩΡΓΙΟΣ,24,ΑΙΟΛΙΚΑ
4,Γ-01328,2006-01-24 00:00:00,ΑΙΟΛΙΚΑ ΠΑΡΚΑ ΠΛΑΤΑΝΟΣ ΑΕ,ΑΔ-01257,2009-06-16 00:00:00,2034-06-16 00:00:00,ΣΤ ΕΛΛΑΔΑΣ,ΕΥΒΟΙΑΣ,ΚΑΡΥΣΤΟΥ,ΜΑΡΜΑΡΙΟΥ,ΠΛΑΤΑΝΟΣ,13.8,ΑΙΟΛΙΚΑ
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98090,Γ-011867,2020-12-20 15:28:51,NKZ ENERGY ΜΟΝΟΠΡΟΣΩΠΗ ΙΔΙΩΤΙΚΗ ΚΕΦΑΛΑΙΟΥΧΙΚΗ ...,ΑΔ-010950,2023-10-09 14:13:22,2048-10-09 14:13:22,"ΘΕΣΣΑΛΙΑΣ,ΣΤΕΡΕΑΣ ΕΛΛΑΔΑΣ","ΜΑΓΝΗΣΙΑΣ,ΦΘΙΩΤΙΔΑΣ","ΑΛΜΥΡΟΥ,ΣΤΥΛΙΔΟΣ","ΑΛΜΥΡΟΥ,ΕΧΙΝΑΙΩΝ",ΓΕΡΑΚΟΒΟΥΝΙ - ΜΕΓΑΣ ΛΑΚΚΟΣ,31.8,ΑΙΟΛΙΚΑ
98091,Γ-014212,2022-06-23 10:54:27.155000,KALERO LIMITED,ΑΔ-010952,2023-10-13 15:22:43,2048-10-13 15:22:43,ΘΕΣΣΑΛΙΑΣ,ΤΡΙΚΑΛΩΝ,ΜΕΤΕΩΡΩΝ,ΑΣΠΡΟΠΟΤΑΜΟΥ,ΡΕΜΑ ΣΚΛΗΝΙΑΣΩΤΙΚΟ,0.49,ΜΥΗΕ
98092,Γ-014371,2022-06-28 17:32:53.453000,ALPHA ΣΥΜΒΟΥΛΟΙ ΙΔΙΩΤΙΚΗ ΚΕΦΑΛΑΙΟΥΧΙΚΗ ΕΤΑΙΡΙΑ,ΑΔ-010953,2023-10-13 15:24:18,2048-10-13 15:24:18,ΘΕΣΣΑΛΙΑΣ,ΤΡΙΚΑΛΩΝ,ΜΕΤΕΩΡΩΝ,ΑΣΠΡΟΠΟΤΑΜΟΥ,ΚΛΕΦΤΟΒΡΥΣΗ - Ρ. ΣΤΑΓΙΑ,0.49,ΜΥΗΕ
98093,Γ-012175,2021-02-03 11:24:16,KSK CAPITAL Ι Κ Ε / KSK CAPITAL,ΑΔ-010954,2023-10-30 10:52:03,2048-10-30 10:52:03,ΘΕΣΣΑΛΙΑΣ,ΚΑΡΔΙΤΣΑΣ,ΚΑΡΔΙΤΣΑΣ,ΙΤΑΜΟΥ,ΟΞΟΥΛΑ,1.06,ΜΥΗΕ


Ερώτηση 1: Πώς ορίζουμε το duplicate γιατί μπορεί κάποιος να καταθέση
Θεωρήσεις

1.   Κάθε αίτηση είναι μοναδική. Δηλαδή δεν μπορώ στην ίδια περιοχή για την ίδια περίοδο ίδια τεχνολογία να έχω και ίδια αίτηση. Για κάθε πάρκο έχει άλλη αίτηση.
2.   Το μοναδικό χαρακτηριστικό που παίρνει ο αιτών εφόσον γίνει δεκτή η αίτηση του είναι ο αριθμός μητρώου ΡΑΕ
3. Επίσης υπάρχουν διάφορες εταιρίες με ποικιλία εξειδίκευσης αν θέλουμε να βγάλουμε χαρακτηριστικά αλλά δημιουρφείται θέμα με τα ονόματα (βλέπε παρακάτω)
4.
5.



In [ ]:
#Drop the duplicate values
f_df = pre_final_df.drop_duplicates(keep='first')
print('after dropping duplicates',len(f_df))
f_df.to_csv('./after_removing_duplicates.csv')

after dropping duplicates 15538


# Μοναδικότητα αίτησης

In [ ]:
vals_count = f_df['ΑΙΤΗΣΗ'].value_counts()
print('non unique',len(vals_count[vals_count!=1]),'unique',(len(vals_count[vals_count==1])),'total',len(vals_count))
print(vals_count.head(5))

non unique 5063 unique 1798 total 6861
ΑΙΤΗΣΗ
Γ-08484     39
Ι-68749     36
I-68749     14
Γ-013138     8
Γ-06887      7
Name: count, dtype: int64


Παρατηρούνται τα εξής

1.   Για διάφορες περιχές/τεχνολογίες και αριθμό μητρώου έχουμε τον ίδιο αριθμός αιτήσεων οπότε δεν μπορεί να θεωρηθεί αξιόπιστο χαρακτηριστικό και πρέπει να πεταχθεί



In [ ]:
test_df_aithsh = f_df.loc[f_df['ΑΙΤΗΣΗ']=='Ι-68749']
print(test_df_aithsh)

        ΑΙΤΗΣΗ ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ  \
4272   Ι-68749         2008-03-05 00:00:00   
4273   Ι-68749         2008-03-05 00:00:00   
4274   Ι-68749         2008-03-05 00:00:00   
4275   Ι-68749         2008-03-05 00:00:00   
4276   Ι-68749         2008-03-05 00:00:00   
4277   Ι-68749         2008-03-05 00:00:00   
4278   Ι-68749         2008-03-05 00:00:00   
4279   Ι-68749         2008-03-05 00:00:00   
4280   Ι-68749         2008-03-05 00:00:00   
4281   Ι-68749         2008-03-05 00:00:00   
4282   Ι-68749         2008-03-05 00:00:00   
4283   Ι-68749         2008-03-05 00:00:00   
4284   Ι-68749         2008-03-05 00:00:00   
4328   Ι-68749         2008-03-05 00:00:00   
4449   Ι-68749         2008-03-05 00:00:00   
9123   Ι-68749         2008-03-05 00:00:00   
12429  Ι-68749         2008-03-05 00:00:00   
12437  Ι-68749         2008-03-05 00:00:00   
12438  Ι-68749         2008-03-05 00:00:00   
12441  Ι-68749         2008-03-05 00:00:00   
12452  Ι-68749         2008-03-05 

# Διερεύνηση μοναδικότητας αριθμού μητρώου αδειών



In [ ]:
vals_count_1 = f_df['AΡΙΘΜΟΣ ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ'].value_counts()
print('non unique',len(vals_count_1[vals_count_1!=1]),'unique',(len(vals_count_1[vals_count_1==1])),'total',len(vals_count_1))
print(vals_count_1.head(10))

non unique 5166 unique 1602 total 6768
AΡΙΘΜΟΣ ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ
00:00:00    8
ΑΔ-07432    7
ΑΔ-06798    7
ΑΔ-06707    7
ΑΔ-06710    7
ΑΔ-06711    7
ΑΔ-06712    7
ΑΔ-06713    7
ΑΔ-06714    7
ΑΔ-08665    7
Name: count, dtype: int64


**ΣΧΟΛΙΑ**
η διαφορά είναι στο


1.   όνομα της εταιρείας
2.   ημερομηνία έκδοση άδειας παραγωγής
ΥΠΟΤΙΘΕΤΑΙ ΟΤΙ ΟΜΩΣ ΕΙΝΑΙ ΜΟΝΑΔΙΚΗ ΓΙΑ ΚΑΘΕ ΝΕΟ ΠΑΡΚΟ ΑΡΑ ΔΕΝ ΜΠΟΡΕΊ ΝΑ ΘΕΩΡΗΘΕΙ ΑΞΙΟΠΙΣΤΗ και πρέπει να πεταχθεί


In [ ]:
test_df_mhtrwo_permits = f_df.loc[f_df['AΡΙΘΜΟΣ ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ']=='ΑΔ-08665']
print(test_df_mhtrwo_permits)

         ΑΙΤΗΣΗ ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ  \
8499   Γ-012268                  06/02/2021   
15994  Γ-012268                  06/02/2021   
21267  Γ-012268                  06/02/2021   
32047  Γ-012268                  06/02/2021   
36874  Γ-012268                  06/02/2021   
42802  Γ-012268                  06/02/2021   
81210  Γ-012268                  06/02/2021   

                                                ΕΤΑΙΡΕΙΑ  \
8499    DEVELOPMENT FARIA GROUP ΙΔΙΩΤΙΚΗ ΚΕΦΑΛΑΙΟΥΧΙΚ...   
15994                                ΠΡΙΜΕΡΑ ΕΝΕΡΤΖΙ Ε Ε   
21267  VELANIDIA 1 ΜΟΝΟΠΡΟΣΩΠΗ Ι.Κ.Ε ΚΑΙ Δ.Τ. VELANID...   
32047   DEVELOPMENT FARIA GROUP ΙΔΙΩΤΙΚΗ ΚΕΦΑΛΑΙΟΥΧΙΚ...   
36874   DEVELOPMENT FARIA GROUP ΙΔΙΩΤΙΚΗ ΚΕΦΑΛΑΙΟΥΧΙΚ...   
42802   DEVELOPMENT FARIA GROUP ΙΔΙΩΤΙΚΗ ΚΕΦΑΛΑΙΟΥΧΙΚ...   
81210                                        ΔΑΝΑΗ ΒΕΜΟΥ   

      AΡΙΘΜΟΣ ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ  \
8499                ΑΔ-08665                   29/06/2021   
15994               

In [ ]:
#FIXING TIMESTAMPS
from collections import Counter
f_df["ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ"] =  pd.to_datetime(f_df["ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ"],format='%Y-%m-%d %H:%M:%S', errors='coerce')
f_df["ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ"] = pd.to_datetime(f_df["ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ"],format='%Y-%m-%d %H:%M:%S', errors='coerce')
f_df["ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ"] = pd.to_datetime(f_df["ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ"],format='%Y-%m-%d %H:%M:%S', errors='coerce')

f_df['ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗ-ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'] = f_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'] - f_df['ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ']
f_df['ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗ-ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'] = f_df['ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗ-ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'].dt.total_seconds().astype(float)
f_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ.-ΛΗΞΗ ΑΔ.ΠΑΡΑΓΩΓΗΣ'] = f_df['ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ'] - f_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ']
f_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ.-ΛΗΞΗ ΑΔ.ΠΑΡΑΓΩΓΗΣ'] = f_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ.-ΛΗΞΗ ΑΔ.ΠΑΡΑΓΩΓΗΣ'].dt.total_seconds().astype(float)
f_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ.-ΛΗΞΗ ΑΔ.ΠΑΡΑΓΩΓΗΣ']=f_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ.-ΛΗΞΗ ΑΔ.ΠΑΡΑΓΩΓΗΣ'].round()/(3600*24*365)

#   Σχόλια για ημερομηνίες


**ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ -ΕΚΔΟΣΗ**
1.   Η πλειοψηφία των τιμών είναι nan 6847
2.   Toδεύτερο μεγαλύτερο νούμερο δείχνει ότι η σχέση μεταξύ αίτηση και έκδωσης είναι 1 χρόνος
Επειδή κυριαρχούν τα nan δεν μπορούμε να βασιστούμε σε αυτήν την στήλη

Απο την άλλη για την έκδωση λήξη παρατηρείται ότι:

1.   Οι περισσότερες τιμές έχουν διαφοράς 25 χρόνια άρα μπορούμε να διορθώσουμε τις nan values ή αρνητικές τιμές που έχουν περαστεί λανθασμένα
επομένβς ημερομηνία έκδοσης είναι αξιόπιστη


In [ ]:
issuance_app_list = f_df['ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗ-ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'].values
issuance_app_list = issuance_app_list/(3600*24*365) # Μετα
issuance_app_list = issuance_app_list.round()
print('COUNTER\n',Counter(issuance_app_list).most_common(),'\n MAX',max(issuance_app_list))
terminate_issuance= f_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ.-ΛΗΞΗ ΑΔ.ΠΑΡΑΓΩΓΗΣ'].values
# terminate_issuance = terminate_issuance/(3600*24*365) # Μετα
terminate_issuance = terminate_issuance.round()
print(terminate_issuance)
count_issuance_app = Counter(issuance_app_list)
print(count_issuance_app.most_common())

COUNTER
 [(1.0, 3460), (2.0, 1870), (0.0, 1510), (3.0, 956), (4.0, 280), (5.0, 143), (6.0, 105), (8.0, 73), (7.0, 59), (9.0, 53), (12.0, 40), (14.0, 37), (10.0, 23), (19.0, 22), (13.0, 14), (11.0, 10), (-15.0, 8), (-8.0, 3), (-16.0, 3), (-11.0, 3), (-5.0, 3), (15.0, 3), (16.0, 3), (-14.0, 2), (-7.0, 2), (17.0, 2), (-17.0, 1), (-9.0, 1), (-19.0, 1), (-10.0, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1), (nan, 1

In [ ]:
import math
nan_count = sum(1 for tup in count_issuance_app.most_common() if math.isnan(tup[0]) and tup[1] == 1)
print("Number of (nan, 1) tuples:", nan_count)

Number of (nan, 1) tuples: 6847


In [ ]:
f_df.isna().sum()

ΑΙΤΗΣΗ                                     1
ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ             6751
ΕΤΑΙΡΕΙΑ                                   1
AΡΙΘΜΟΣ ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ                     1
ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ            4724
ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ           3019
ΠΕΡΙΦΕΡΕΙΑ                                 1
ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ                       1
ΔΗΜΟΣ                                      1
ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ                           5
ΘΕΣΗ                                       1
ΜΕΓΙΣΤΗ ΙΣΧΥΣ (MW)                         1
ΤΕΧΝΟΛΟΓΙΑ                                 1
ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗ-ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ    6847
ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ.-ΛΗΞΗ ΑΔ.ΠΑΡΑΓΩΓΗΣ       4739
dtype: int64

In [ ]:
count_terminate_issuance = Counter(terminate_issuance) # πόσα χρόνια μεταξύ λ΄ήξης και έναρξης της επεξεργασίας
print(count_terminate_issuance.most_common(30))
nan_count_terminated = sum(1 for tup in count_terminate_issuance.most_common() if math.isnan(tup[0]) and tup[1] == 1)
print('Nan values',nan_count_terminated)

[(25.0, 10497), (15.0, 68), (40.0, 48), (0.0, 19), (50.0, 17), (37.0, 16), (38.0, 15), (19.0, 13), (16.0, 12), (39.0, 11), (20.0, 9), (27.0, 8), (-75.0, 7), (18.0, 7), (30.0, 7), (36.0, 6), (24.0, 5), (43.0, 5), (17.0, 5), (34.0, 4), (46.0, 3), (14.0, 3), (44.0, 2), (28.0, 2), (22.0, 2), (nan, 1), (23.0, 1), (nan, 1), (nan, 1), (nan, 1)]
Nan values 4739


In [ ]:
#Αν και οι 2 ημερομηνείς είναι 0 τότε δεν μπορώ να κάνω κάτι
same_time_nan = f_df.index[(f_df['ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ'].isna())&(f_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'].isna())].tolist()
print(len(same_time_nan))
selected_nan_time_df = f_df[f_df.index.isin(same_time_nan)]
clear_time_df = f_df.drop(same_time_nan)
clear_time_df = clear_time_df.drop_duplicates(keep='first')
clear_time_df.reset_index(drop=True,inplace=True)
clear_time_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ.-ΛΗΞΗ ΑΔ.ΠΑΡΑΓΩΓΗΣ'] = clear_time_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ.-ΛΗΞΗ ΑΔ.ΠΑΡΑΓΩΓΗΣ'].round()
index_minus_75= clear_time_df.index[clear_time_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ.-ΛΗΞΗ ΑΔ.ΠΑΡΑΓΩΓΗΣ']==-75.0]
index_0 =  clear_time_df.index[clear_time_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ.-ΛΗΞΗ ΑΔ.ΠΑΡΑΓΩΓΗΣ']==0.0]

3004


some cleaning for negative values

In [ ]:
clear_time_df_copy = clear_time_df.copy()
for inda in index_0:
    clear_time_df_copy.loc[inda,'ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ'] = clear_time_df_copy.loc[inda,'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'] + pd.DateOffset(years=25)

some_results_0 = (clear_time_df_copy['ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ'] - clear_time_df_copy['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ']).dt.total_seconds().astype(float)
some_results_0 = some_results_0/(3600*24*365)
some_results_0=some_results_0.round()
clear_time_df_copy['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ.-ΛΗΞΗ ΑΔ.ΠΑΡΑΓΩΓΗΣ'] = some_results_0

for inda in index_minus_75:
    clear_time_df_copy.loc[inda,'ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ'] = clear_time_df_copy.loc[inda,'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'] + pd.DateOffset(years=25)
some_results_75 = (clear_time_df_copy['ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ'] - clear_time_df_copy['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ']).dt.total_seconds().astype(float)
some_results_75 = some_results_75/(3600*24*365)
some_results_75=some_results_75.round()
clear_time_df_copy['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ.-ΛΗΞΗ ΑΔ.ΠΑΡΑΓΩΓΗΣ'] = some_results_75
index_submission_values = clear_time_df_copy.index[clear_time_df_copy['ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ'].isna()]
index_termination_values =clear_time_df_copy.index[clear_time_df_copy['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'].isna()]
# print(index_termination_values)
c_t_df = clear_time_df_copy.copy()
for s_inde in index_submission_values:
    c_t_df.loc[s_inde,'ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ'] = c_t_df.loc[s_inde,'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'] + pd.DateOffset(years=25)

for ter_inde in index_termination_values:
    c_t_df.loc[ter_inde,'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'] = c_t_df.loc[ter_inde,'ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ'] - pd.DateOffset(years=25)

some_results_final = (c_t_df['ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ'] - c_t_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ']).dt.total_seconds().astype(float)
some_results_final = some_results_final/(3600*24*365)
some_results_final=some_results_final.round()
c_t_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ.-ΛΗΞΗ ΑΔ.ΠΑΡΑΓΩΓΗΣ'] = some_results_final
index_minus_2 = c_t_df.index[c_t_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ.-ΛΗΞΗ ΑΔ.ΠΑΡΑΓΩΓΗΣ']==-2.0]
# print(c_t_df.loc[index_minus_2])
for inda in index_minus_2:
    c_t_df.loc[inda,'ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ'] = c_t_df.loc[inda,'ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'] + pd.DateOffset(years=25)
some_results_2 = (c_t_df['ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ'] - c_t_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ']).dt.total_seconds().astype(float)
some_results_2 = some_results_2/(3600*24*365)
some_results_2=some_results_2.round()
c_t_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ.-ΛΗΞΗ ΑΔ.ΠΑΡΑΓΩΓΗΣ'] = some_results_2

In [ ]:
print(c_t_df.isna().any())
final_count = Counter(c_t_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ.-ΛΗΞΗ ΑΔ.ΠΑΡΑΓΩΓΗΣ'].values)
print(final_count.most_common(30))


ΑΙΤΗΣΗ                                  False
ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗΣ ΑΙΤΗΣΗΣ              True
ΕΤΑΙΡΕΙΑ                                False
AΡΙΘΜΟΣ ΜΗΤΡΩΟΥ ΑΔΕΙΩΝ                  False
ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ            False
ΗΜΕΡΟΜΗΝΙΑ ΛΗΞΗΣ ΑΔ.ΠΑΡΑΓΩΓΗΣ           False
ΠΕΡΙΦΕΡΕΙΑ                              False
ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ                    False
ΔΗΜΟΣ                                   False
ΔΗΜΟΤΙΚΗ ΕΝΟΤΗΤΑ                         True
ΘΕΣΗ                                    False
ΜΕΓΙΣΤΗ ΙΣΧΥΣ (MW)                      False
ΤΕΧΝΟΛΟΓΙΑ                              False
ΗΜΕΡΟΜΗΝΙΑ ΥΠΟΒΟΛΗ-ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ     True
ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ.-ΛΗΞΗ ΑΔ.ΠΑΡΑΓΩΓΗΣ       False
dtype: bool
[(25.0, 12259), (15.0, 68), (40.0, 48), (50.0, 17), (37.0, 16), (38.0, 15), (19.0, 13), (16.0, 12), (39.0, 11), (20.0, 9), (27.0, 8), (18.0, 7), (30.0, 7), (36.0, 6), (24.0, 5), (43.0, 5), (17.0, 5), (34.0, 4), (46.0, 3), (14.0, 3), (44.0, 2), (28.0, 2), (22.0, 2), (23.0, 1), (52.0, 1), (47.0, 1),

In [ ]:
c_t_df.drop_duplicates(keep='first',inplace=True)
c_t_df.reset_index(drop=True,inplace=True)
print(len(c_t_df))
# c_t_df.to_csv('./timestamp_correction.csv')

12490


In [ ]:
# v_1_df = pd.read_csv('./timestamp_correction.csv',index_col=False)
v_1_df = c_t_df
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΣΤ ΕΛΛΑΔΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΠΕΛΟΠΟΝΝΗΣΟΥ ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΠΕΛΟΠΟΝΝΗΣΟΥ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΣΤΕΡΕΑΣ ΕΛΛΑΔΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΔΥΤΙΚΗΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='Κ ΜΑΚΕΔΟΝΙΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΑΝ ΜΑΚΕΔΟΝΙΑΣ ΘΡΑΚΗΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΑΝΑΤΟΛΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ ΚΑΙ ΘΡΑΚΗΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='Δ ΜΑΚΕΔΟΝΙΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΔΥΤΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='Δ ΕΛΛΑΔΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΔΥΤΙΚΗΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΑΝ. ΜΑΚΕΔΟΝΙΑΣ & ΘΡΑΚΗΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΑΝΑΤΟΛΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ ΚΑΙ ΘΡΑΚΗΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='Ν ΑΙΓΑΙΟΥ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΝΟΤΙΟΥ ΑΙΓΑΙΟΥ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='Β ΑΙΓΑΙΟΥ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΑΝΑΤΟΛΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ & ΘΡΑΚΗΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΑΝΑΤΟΛΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ ΚΑΙ ΘΡΑΚΗΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΙΟΝΙΩΝ ΝΗΣΩΝ','ΠΕΡΙΦΕΡΕΙΑ'] =  'ΙΟΝΙΩΝ ΝΗΣΙΩΝ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΘΕΣΣΑΛΙΑΣ,ΣΤΕΡΕΑΣ ΕΛΛΑΔΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΘΕΣΣΑΛΙΑΣ ΚΑΙ ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΔΥΤΙΚΗΣ ΕΛΛΑΔΟΣ & ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΔΥΤΙΚΗΣ ΕΛΛΑΔΟΣ ΚΑΙ ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΑΝΑΤΟΛΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ ΘΡΑΚΗΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΑΝΑΤΟΛΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ ΚΑΙ ΘΡΑΚΗΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΘΕΣΣΑΛΙΑΣ,ΗΠΕΙΡΟΥ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΘΕΣΣΑΛΙΑΣ ΚΑΙ ΗΠΕΙΡΟΥ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΔΥΤΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ,ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ ΚΑΙ ΔΥΤΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΑΤΤΙΚΗΣ,ΣΤΕΡΕΑΣ ΕΛΛΑΔΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΑΤΤΙΚΗΣ ΚΑΙ ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΔΥΤΙΚΗΣ ΕΛΛΑΔΟΣ ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΔΥΤΙΚΗΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΗΠΕΙΡΟΥ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΗΠΕΙΡΟΥ '
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΘΕΣΣΑΛΙΑΣ - ΣΤΕΡΕΑΣ ΕΛΛΑΔΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΘΕΣΣΑΛΙΑΣ ΚΑΙ ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΘΕΣΣΑΛΙΑΣ,ΔΥΤΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΘΕΣΣΑΛΙΑΣ ΚΑΙ ΔΥΤΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΠΕΛΟΠΟΝΝΗΣΟΥ,ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΠΕΛΟΠΟΝΝΗΣΟΥ ΚΑΙ ΔΥΤΙΚΗΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ  -  ΑΤΤΙΚΗΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΑΤΤΙΚΗΣ ΚΑΙ ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ - ΘΕΣΣΑΛΙΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΘΕΣΣΑΛΙΑΣ ΚΑΙ ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΑΝ. ΜΑΚΕΔΟΝΙΑΣ & ΘΡΑΚΗΣ,ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΑΝΑΤΟΛΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ ΚΑΙ ΘΡΑΚΗΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΔΥΤΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ,ΘΕΣΣΑΛΙΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΘΕΣΣΑΛΙΑΣ ΚΑΙ ΔΥΤΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΠΕΛΟΠΟΝΝΗΣΟΥ ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΠΕΛΟΠΟΝΝΗΣΟΥ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='Δ. ΜΑΚΕΔΟΝΙΑΣ - Κ. ΜΑΚΕΔΟΝΙΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΠΕΛΟΠΟΝΝΗΣΟΥ, ΑΤΤΙΚΗΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΑΤΤΙΚΗΣ ΚΑΙ ΠΕΛΟΠΟΝΝΗΣΟΥ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΣΤΕΡΕΑ ΕΛΛΑΔΑ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ & ΣΤΕΡΕΑΣ ΕΛΛΑΔΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΔΥΤΙΚΗΣ ΕΛΛΑΔΟΣ ΚΑΙ ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΗΠΕΙΡΟΥ - ΘΕΣΣΑΛΙΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΘΕΣΣΑΛΙΑΣ ΚΑΙ ΗΠΕΙΡΟΥ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ,ΣΤΕΡΕΑΣ ΕΛΛΑΔΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΔΥΤΙΚΗΣ ΕΛΛΑΔΟΣ ΚΑΙ ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ,ΑΝ. ΜΑΚΕΔΟΝΙΑΣ & ΘΡΑΚΗΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΑΝΑΤΟΛΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ ΚΑΙ ΘΡΑΚΗΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΑΤΤΙΚΗΣ ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΑΤΤΙΚΗΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΠΕΛΟΠΟΝΝΗΣΟΥ - ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΠΕΛΟΠΟΝΝΗΣΟΥ ΚΑΙ ΔΥΤΙΚΗΣ ΕΛΛΑΔOΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ - ΘΕΣΣΑΛΙΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΘΕΣΣΑΛΙΑΣ ΚΑΙ ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='Δ ΜΑΚΕΔΟΝΙΑΣ - ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΜΑΚΕΔΟΝΙΑΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΔΥΤΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ,ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΜΑΚΕΔΟΝΙΑΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='Δ. ΕΛΛΑΔΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΔΥΤΙΚΗΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΑΤΤΙΚΗΣ & ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΑΤΤΙΚΗΣ ΚΑΙ ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΠΕΛΟΠΟΝΝΗΣΟΥ - ΑΤΤΙΚΗΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΑΤΤΙΚΗΣ ΚΑΙ ΠΕΛΟΠΟΝΝΗΣΟΥ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΣΤΕΡΕΑΣ ΕΛΛΑΔΑΣ - ΘΕΣΣΑΛΙΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΘΕΣΣΑΛΙΑΣ ΚΑΙ ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ & ΘΕΣΣΑΛΙΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΘΕΣΣΑΛΙΑΣ ΚΑΙ ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='Δ ΜΑΚΕΔΟΝΙΑΣ - Κ ΜΑΚΕΔΟΝΙΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΑΤΤΙΚΗΣ & ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΑΤΤΙΚΗΣ ΚΑΙ ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΔΥΤΙΚΗΣ ΕΛΛΑΔΟΣ -  ΚΕΝΤΡΙΚΗΣ  ΜΑΚΕΔΟΝΙΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΜΑΚΕΔΟΝΙΑΣ ΚΑΙ ΔΥΤΙΚΗΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΘΕΣΣΑΛΙΑΣ & ΔΥΤΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΘΕΣΣΑΛΙΑΣ ΚΑΙ ΔΥΤΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΘΕΣΣΑΛΙΑΣ & ΔΥΤΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ ','ΠΕΡΙΦΕΡΕΙΑ'] = 'ΘΕΣΣΑΛΙΑΣ ΚΑΙ ΔΥΤΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΔΥΤΙΚΗΣ ΕΛΛΑΔΑΣ,ΠΕΛΟΠΟΝΝΗΣΟΥ','ΠΕΡΙΦΕΡΕΙΑ']='ΠΕΛΟΠΟΝΝΗΣΟΥ ΚΑΙ ΔΥΤΙΚΗΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΑΝ. ΜΑΚΕΔΟΝΙΑΣ & ΘΡΑΚΗΣ ','ΠΕΡΙΦΕΡΕΙΑ']='ΑΝΑΤΟΛΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ ΚΑΙ ΘΡΑΚΗΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΑΝΑΤΟΛΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ & ΘΡΑΚΗΣ ','ΠΕΡΙΦΕΡΕΙΑ']='ΑΝΑΤΟΛΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ ΚΑΙ ΘΡΑΚΗΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΣΤΕΡΕΑΣ ΕΛΛΑΔΑΣ,ΘΕΣΣΑΛΙΑΣ','ΠΕΡΙΦΕΡΕΙΑ']= 'ΘΕΣΣΑΛΙΑΣ ΚΑΙ ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ ','ΠΕΡΙΦΕΡΕΙΑ']= 'ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ'
v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ']=='ΠΕΛΟΠΟΝΝΗΣΟΥ             ','ΠΕΡΙΦΕΡΕΙΑ']= 'ΠΕΛΟΠΟΝΝΗΣΟΥ'

In [ ]:
exam_regios = ['ΠΕΛΟΠΟΝΝΗΣΟΥ ΚΑΙ ΔΥΤΙΚΗΣ ΕΛΛΑΔΟΣ',
 'ΜΑΚΕΔΟΝΙΑΣ',  'ΜΑΚΕΔΟΝΙΑΣ ΚΑΙ ΔΥΤΙΚΗΣ ΕΛΛΑΔΟΣ',
 'ΘΕΣΣΑΛΙΑΣ ΚΑΙ ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ','ΑΤΤΙΚΗΣ ΚΑΙ ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ',
 'ΘΕΣΣΑΛΙΑΣ ΚΑΙ ΔΥΤΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ','ΔΥΤΙΚΗΣ ΕΛΛΑΔΟΣ ΚΑΙ ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ',
 'ΑΤΤΙΚΗΣ ΚΑΙ ΠΕΛΟΠΟΝΝΗΣΟΥ','ΘΕΣΣΑΛΙΑΣ ΚΑΙ ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ',
 'ΠΕΛΟΠΟΝΝΗΣΟΥ ΚΑΙ ΔΥΤΙΚΗΣ ΕΛΛΑΔOΣ', 'ΘΕΣΣΑΛΙΑΣ ΚΑΙ ΗΠΕΙΡΟΥ',
 'ΜΑΚΕΔΟΝΙΑΣ ΚΑΙ ΘΡΑΚΗΣ']
find_cities = v_1_df.loc[v_1_df['ΠΕΡΙΦΕΡΕΙΑ'].isin(exam_regios)] # I will drop these values beacuse they belong to multi regions
print(len(find_cities))
v_2_df = v_1_df.loc[~v_1_df['ΠΕΡΙΦΕΡΕΙΑ'].isin(exam_regios)] #Select only the mandatory regions regions
print(len(v_2_df))
# drop_geo_columns = ['ΠΕΡΙΦΕΡΕΙΑΚΗ ΕΝΟΤΗΤΑ','ΔΗΜΟΣ ', 'ΘΕΣΗ','Unnamed: 0','ΑΙΤΗΣΗ']
# v_2_df.drop(columns=drop_geo_columns,inplace=True)
# print(len(v_2_df))
# v_3_df =v_2_df.drop_duplicates(keep='first')
# v_3_df.drop(columns = ['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ.-ΛΗΞΗ ΑΔ.ΠΑΡΑΓΩΓΗΣ'],inplace=True)
# print(len(v_3_df))

72
12418


In [ ]:
print(len(v_4_df['ΤΕΧΝΟΛΟΓΙΑ'].unique()))
v_4_df.loc[v_4_df['ΤΕΧΝΟΛΟΓΙΑ']=='ΒΙΟΜΑΖΑ - ΒΙΟΑΕΡΙΟ','ΤΕΧΝΟΛΟΓΙΑ']= 'ΒΙΟΜΑΖΑ'
v_4_df.loc[v_4_df['ΤΕΧΝΟΛΟΓΙΑ']=='ΒΙΟΑΕΡΙΟ','ΤΕΧΝΟΛΟΓΙΑ']= 'ΒΙΟΜΑΖΑ'
v_4_df.loc[v_4_df['ΤΕΧΝΟΛΟΓΙΑ']=='ΒΙΟΜΑΖΑ-ΒΙΟΑΕΡΙΟ','ΤΕΧΝΟΛΟΓΙΑ']= 'ΒΙΟΜΑΖΑ'
v_4_df.loc[v_4_df['ΤΕΧΝΟΛΟΓΙΑ']=='ΑΙΟΛΙΚΑ ','ΤΕΧΝΟΛΟΓΙΑ']= 'ΑΙΟΛΙΚΑ'
v_4_df.loc[v_4_df['ΤΕΧΝΟΛΟΓΙΑ']=='ΑΙΟΛΙΚΑ ','ΤΕΧΝΟΛΟΓΙΑ']= 'ΑΙΟΛΙΚΑ'
v_4_df.loc[v_4_df['ΤΕΧΝΟΛΟΓΙΑ']=='ΑΙΟΛΙΚΑ  ','ΤΕΧΝΟΛΟΓΙΑ']= 'ΑΙΟΛΙΚΑ'
v_4_df.loc[v_4_df['ΤΕΧΝΟΛΟΓΙΑ']=='ΘΑΛ. ΑΙΟΛΙΚΑ ','ΤΕΧΝΟΛΟΓΙΑ']= 'ΑΙΟΛΙΚΑ'
v_4_df.loc[v_4_df['ΤΕΧΝΟΛΟΓΙΑ']=='ΘΑΛ. ΑΙΟΛΙΚΑ','ΤΕΧΝΟΛΟΓΙΑ']= 'ΑΙΟΛΙΚΑ'
v_4_df.loc[v_4_df['ΤΕΧΝΟΛΟΓΙΑ']=='ΓΕΩΘΕΡΜΙΑ','ΤΕΧΝΟΛΟΓΙΑ']= 'ΑΙΟΛΙΚΑ'
v_4_df.loc[v_4_df['ΤΕΧΝΟΛΟΓΙΑ']=='ΒΙΟΜΑΖΑ-ΚΑΥΣΗ','ΤΕΧΝΟΛΟΓΙΑ']= 'ΒΙΟΜΑΖΑ'
print(len(v_4_df['ΤΕΧΝΟΛΟΓΙΑ'].unique()))
print(len(v_4_df))
v_4_df.drop_duplicates(keep='first',inplace=True)
print(len(v_4_df))
v_4_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'] = pd.to_datetime(v_4_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'])
#Generate the different time characteristics
v_4_df['day_week']  = v_4_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'].dt.dayofweek
v_4_df['day_year'] = v_4_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'].dt.day_of_year
v_4_df['month'] = v_4_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'].dt.month
v_4_df['year'] = v_4_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'].dt.year
v_4_df['quarter'] = v_4_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'].dt.quarter
v_4_df['week_year'] = v_4_df['ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ'].dt.isocalendar().week
v_4_df.sort_values(by='ΗΜΕΡΟΜΗΝΙΑ ΕΚΔ. ΑΔ.ΠΑΡΑΓΩΓΗΣ',inplace=True)
v_4_df.reset_index(inplace=True,drop=True)
v_4_df['ΠΕΡΙΦΕΡΕΙΑ'].unique()
mapping_region ={
    'ΣΤΕΡΕΑΣ ΕΛΛΑΔΟΣ':'ΛΑΜΙΑ',
    'ΝΟΤΙΟΥ ΑΙΓΑΙΟΥ':'ΕΡΜΟΥΠΟΛΗ',
    'ΚΡΗΤΗΣ':'ΗΡΑΚΛΕΙΟ',
    'ΒΟΡΕΙΟΥ ΑΙΓΑΙΟΥ':'ΜΥΤΙΛΗΝΗ',
    'ΚΕΝΤΡΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ':'ΘΕΣΣΑΛΟΝΙΚΗ',
    'ΘΕΣΣΑΛΙΑΣ':'ΛΑΡΙΣΑ',
    'ΔΥΤΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ':'ΚΟΖΑΝΗ',
    'ΗΠΕΙΡΟΥ ':'ΙΩΑΝΝΙΝΑ',
    'ΑΝΑΤΟΛΙΚΗΣ ΜΑΚΕΔΟΝΙΑΣ ΚΑΙ ΘΡΑΚΗΣ':'ΚΟΜΟΤΗΝΗ',
    'ΔΥΤΙΚΗΣ ΕΛΛΑΔΟΣ':'ΠΑΤΡΑ',
    'ΠΕΛΟΠΟΝΝΗΣΟΥ':'ΤΡΙΠΟΛΗ',
    'ΑΤΤΙΚΗΣ':'ΑΘΗΝΑ',
    'ΙΟΝΙΩΝ ΝΗΣΙΩΝ':'ΚΕΡΚΥΡΑ'}
v_4_df['ΠΟΛΗ'] = v_4_df['ΠΕΡΙΦΕΡΕΙΑ'].map(mapping_region)